In [ ]:
#|default_exp items

# Items

> Utilities for fetching item catalogs

In [ ]:
#| eval: false
#| hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import requests
import json
from fastcore.all import L
import pandas as pd
from typing import Any
from urllib.parse import urlparse
from pathlib import Path
import numpy as np
import re
from shapely.geometry import Polygon, box
import geopandas as gpd

In [ ]:
#| hide
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',120)

In [ ]:
#| export
from ntlights_damage_assessment.catalogs import *

In [ ]:
%%time
#| eval: false
catalogs = get_item_catalogs()

CPU times: user 25.3 ms, sys: 7.56 ms, total: 32.8 ms
Wall time: 1.5 s


In [ ]:
#| eval: false
catalog1805 = catalogs[catalogs.yearmonth == '201805']

In [ ]:
#| eval: false
href1805 = catalog1805.iloc[0].href; href1805

'https://globalnightlight.s3.amazonaws.com/npp_201805/npp_201805_catalog.json'

In [ ]:
%%time
#| eval: false
data1805 = get_data(href1805)

CPU times: user 48.6 ms, sys: 8.18 ms, total: 56.8 ms
Wall time: 5.57 s


In [ ]:
#| eval:false
df1805 = make_df(data1805,rel='item')

In [ ]:
#| export

def get_item_href(href,folder, baseurl):
    return f'{baseurl}/{folder}{href[1:]}'

In [ ]:
assert get_item_href('./SVDNB_npp_d20180501_t0004098_e0009502_b33718_c20180501060951522600_noac_ops.rade9.co.json',
              '201805',
              'https://globalnightlight.s3.amazonaws.com') == 'https://globalnightlight.s3.amazonaws.com/201805/SVDNB_npp_d20180501_t0004098_e0009502_b33718_c20180501060951522600_noac_ops.rade9.co.json'

In [ ]:
#| export
def split_href_type(href, first=True):
    parts = href[2:].split('.')
    return parts[0] if first else '.'.join(parts[1:])
    

In [ ]:
#| export
def transform_items_df(items_df, folder, baseurl):
    items_df.drop(columns=['rel','type'], inplace=True)
    items_df['item_href'] = items_df.href.apply(get_item_href, folder=folder, baseurl=baseurl)
    items_df['stem'] = items_df.href.apply(split_href_type)
    items_df['suffix'] = items_df.href.apply(split_href_type,first=False)
    items_df['base_url'] = baseurl
    items_df['folder'] = folder
    return items_df